In [ ]:
import os
if not os.path.exists('bookingchallenge.zip'):
  !wget https://shoutout.wix.com/so/e7NOQRe3s/c?w=Yf6RfAsxuTZ-iwMp4ctugpiHMNm6tqV23NoKlVp3LI4.eyJ1IjoiaHR0cHM6Ly9tZWRpYS53aXguY29tL2FyY2hpdmVzLzMwOTFiZV85ZjI0NWM3ZjY0YWY0ZWQyODMxOGE3ZWI5MGNlMjk0Yy56aXAiLCJyIjoiZTE2YjIwZjItZjE5Zi00MDRmLTAzMDYtYjRlYTFlZjM2NjgzIiwibSI6Im1haWwiLCJjIjoiYzYzZDdjNGYtNzMzMy00ZGM2LWJlMWItODdiZTc5NjQ3NThiIn0 -O bookingchallenge.zip
  !unzip bookingchallenge.zip
  !wget https://shoutout.wix.com/so/c4NRCVgeV/c?w=P0QHVGbLhSfHoFBEanWqgTNMNDnH2kFiVG8z8jnuL6U.eyJ1IjoiaHR0cHM6Ly8wMzUxMDVmNy1hZTMyLTQ3YjYtYTI1Yi04N2FmNzkyNGM3ZWEudXNyZmlsZXMuY29tL2FyY2hpdmVzLzMwOTFiZV84Mzc4NGYzMzA1NWM0NzkxYWUyYjY4ODdlMTFkZGYwOS56aXA_ZG49QktOR3Rlc3RkYXRhV1NETTIwMjEuemlwIiwiciI6IjAzMzQzYjU3LTQ5MGYtNDk3OS02NGFjLTZjZTJlYjBiZDU0NyIsIm0iOiJtYWlsIiwiYyI6ImM2M2Q3YzRmLTczMzMtNGRjNi1iZTFiLTg3YmU3OTY0NzU4YiJ9 -O testdata.zip
  !unzip -n testdata.zip  
  !pip install adabelief-tf==0.2.0

if not os.path.exists('./drive/My Drive/predictions/'):
  !mkdir ./drive/My\ Drive/predictions/

import time
T = int(time.time())
path = './drive/My Drive/predictions/sub_%s/'%T
os.mkdir(path)
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.metrics import sparse_top_k_categorical_accuracy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from gensim.models import Word2Vec, FastText
from gensim.models import KeyedVectors

from sklearn import preprocessing
from tensorflow.keras.layers import Embedding
import sys
import warnings
warnings.simplefilter("ignore")

from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM, Dropout, GlobalAveragePooling1D, Conv1D, MaxPooling1D, add, BatchNormalization, Lambda
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from gensim.models import Word2Vec

from adabelief_tf import AdaBeliefOptimizer
from sklearn.model_selection import KFold


--2021-01-22 17:16:24--  https://shoutout.wix.com/so/e7NOQRe3s/c?w=Yf6RfAsxuTZ-iwMp4ctugpiHMNm6tqV23NoKlVp3LI4.eyJ1IjoiaHR0cHM6Ly9tZWRpYS53aXguY29tL2FyY2hpdmVzLzMwOTFiZV85ZjI0NWM3ZjY0YWY0ZWQyODMxOGE3ZWI5MGNlMjk0Yy56aXAiLCJyIjoiZTE2YjIwZjItZjE5Zi00MDRmLTAzMDYtYjRlYTFlZjM2NjgzIiwibSI6Im1haWwiLCJjIjoiYzYzZDdjNGYtNzMzMy00ZGM2LWJlMWItODdiZTc5NjQ3NThiIn0
Resolving shoutout.wix.com (shoutout.wix.com)... 185.230.60.168
Connecting to shoutout.wix.com (shoutout.wix.com)|185.230.60.168|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.wix.com/archives/3091be_9f245c7f64af4ed28318a7eb90ce294c.zip [following]
--2021-01-22 17:16:25--  https://media.wix.com/archives/3091be_9f245c7f64af4ed28318a7eb90ce294c.zip
Resolving media.wix.com (media.wix.com)... 34.102.176.152
Connecting to media.wix.com (media.wix.com)|34.102.176.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16698079 (16M) [application/zip]
Saving to: ‘bookingchallenge.z

In [ ]:
data = pd.read_csv('./booking_train_set.csv')
data_test = pd.read_csv('./booking_test_set.csv')
data_test = data_test.sort_values(by=['utrip_id','checkin'])

N_city = data.city_id.nunique()
N_hotel = data.hotel_country.nunique()
N_affi = data.affiliate_id.nunique()

# 改变城市id的范围
le = preprocessing.LabelEncoder()
data['city_id'] = le.fit_transform(data['city_id'].values)
data['city_id'] = data['city_id'].astype(str)

le_hotel = preprocessing.LabelEncoder()
data['hotel_country'] = le_hotel.fit_transform(data['hotel_country'].values)
data['hotel_country'] = data['hotel_country'].astype(str)

# 去除长度为1的序列
data['Size'] = data.utrip_id.map(data.groupby('utrip_id').agg('size'))
data = data[data.Size>1]

data['time'] = data.checkin.apply(lambda x: int(x[:4]))
data = data[data.time!=2015] # 去掉时间上的outlier

data_test['city_id'] = data_test.city_id.apply(lambda x: 56430 if x==0 else x)
data_test = data_test.fillna('Urkesh')
data_test['Size'] = data_test.utrip_id.map(data_test.groupby('utrip_id').agg('size'))

data_test['city_id'] = le.transform(data_test['city_id'].values)
data_test['city_id'] = data_test['city_id'].astype(str)

data_test['hotel_country'] = le_hotel.transform(data_test['hotel_country'].values)
data_test['hotel_country'] = data_test['hotel_country'].astype(str)

In [ ]:
N = data.user_id.nunique()

data_train = data     # Train dataset
data_test = data_test   # Test dataset

def trans_sincos(df, name='week', MAX=7):
  df[name+'_sin'] = np.sin(np.pi*df[name].values/MAX)
  df[name+'_cos'] = np.cos(np.pi*df[name].values/MAX)
  return df 

def extract_time(df):
  df['dayofweek_in'] = pd.to_datetime(df.checkin).dt.dayofweek.values
  df['dayofmonth_in'] = pd.to_datetime(df.checkin).dt.day.values
  df['dayofweek_out'] = pd.to_datetime(df.checkout).dt.dayofweek.values
  df['dayofmonth_out'] = pd.to_datetime(df.checkout).dt.day.values
  df['month_in'] = pd.to_datetime(df.checkin).dt.month.values
  df['month_out'] = pd.to_datetime(df.checkout).dt.month.values

  df = trans_sincos(df, name='dayofweek_in', MAX=7)
  df = trans_sincos(df, name='dayofmonth_in', MAX=30)
  df = trans_sincos(df, name='dayofweek_out', MAX=7)
  df = trans_sincos(df, name='dayofmonth_out', MAX=30)
  df = trans_sincos(df, name='month_in', MAX=12)
  df = trans_sincos(df, name='month_out', MAX=12)

  df['duration'] = (pd.to_datetime(df.checkout)-pd.to_datetime(df.checkin)).dt.days.values
  df[df.duration>19]['duration']=19

  return df

def get_additionnal_features(data_train, data_test):
  feat_names = ['duration',  'month_in_sin', 'month_out_sin', 'month_in_cos', 'month_out_cos', 'Size',
                'dayofweek_in_cos', 'dayofmonth_in_cos', 'dayofweek_in_sin', 'dayofmonth_in_sin', 
                'dayofweek_out_cos', 'dayofmonth_out_cos', 'dayofweek_out_sin', 'dayofmonth_out_sin'
          ]

  X_train_duration = data_train.groupby('utrip_id')[feat_names].nth(-1).values
  X_test_duration = data_test.groupby('utrip_id')[feat_names].nth(-1).values

  X_train_duration = np.hstack([X_train_duration, data_train.groupby('utrip_id')['duration'].sum().values.reshape((-1,1))])
  X_test_duration = np.hstack([X_test_duration, data_test.groupby('utrip_id')['duration'].sum().values.reshape((-1,1))])

  X_train_affi = data_train.groupby('utrip_id').affiliate_id.last().values
  X_test_affi = data_test.groupby('utrip_id').affiliate_id.last().values

  X_train_features = pd.get_dummies(data_train.groupby('utrip_id').device_class.last()).values
  X_test_features = pd.get_dummies(data_test.groupby('utrip_id').device_class.last()).values

  X_train_features2 = pd.get_dummies(data_train.groupby('utrip_id').booker_country.last()).values
  X_test_features2 = pd.get_dummies(data_test.groupby('utrip_id').booker_country.last()).values

  X_train_features = np.hstack([X_train_features, X_train_features2, X_train_duration, X_train_affi.reshape((-1,1))])
  X_test_features = np.hstack([X_test_features, X_test_features2, X_test_duration, X_test_affi.reshape((-1,1))])

  return X_train_features, X_test_features

data_train = extract_time(data_train)
data_test = extract_time(data_test)

X_train_features, X_test_features = get_additionnal_features(data_train, data_test)

In [ ]:
def getdata():
  X_train = data_train.groupby('utrip_id').city_id.apply(lambda x: list(list(x)[:-1])).values
  X_test = data_test.groupby('utrip_id').city_id.apply(lambda x: list(list(x)[:-1])).values

  X_train_hotel = data_train.groupby('utrip_id').hotel_country.apply(lambda x: list(list(x)[:-1])).values
  X_test_hotel = data_test.groupby('utrip_id').hotel_country.apply(lambda x: list(list(x)[:-1])).values

  X_train0 = data_train.groupby('utrip_id').city_id.apply(lambda x: list(list(x))).values
  X_test0 = data_test.groupby('utrip_id').city_id.apply(lambda x: list(list(x))).values

  X_train0_hotel = data_train.groupby('utrip_id').hotel_country.apply(lambda x: list(list(x))).values

  y_train = data_train.groupby('utrip_id').city_id.agg('last').astype(int).values
  y_test = data_test.groupby('utrip_id').city_id.agg('last').astype(int).values

  y_train_hotel = data_train.groupby('utrip_id').hotel_country.agg('last').astype(int).values
  y_test_hotel = data_test.groupby('utrip_id').hotel_country.agg('last').astype(int).values

  return X_train, X_test, X_train_hotel, X_test_hotel, X_train0, X_train0_hotel, y_train, y_test, y_train_hotel, y_test_hotel, X_test0

def remove_last_row(df):
  keep_index = np.concatenate([df.utrip_id.values[:-1]==df.utrip_id.values[1:], np.array([False])])
  df = df[keep_index]
  df['Size'] = df.utrip_id.map(df.groupby('utrip_id').user_id.size())
  df = df[df.Size>1]

  return df

def produce_extra(X_train, X_test, X_train_hotel_extra, X_test_hotel_extra):
  X_train_len = np.array([len(i) for i in X_train])
  X_train_extra = X_train[X_train_len>1]

  X_train_hotel_extra_len = np.array([len(i) for i in X_train_hotel_extra])
  X_train_hotel_extra = X_train_hotel_extra[X_train_hotel_extra_len>1]

  y_train_extra = np.array([int(i[-1]) for i in X_train_extra])
  y_train_hotel_extra = np.array([int(i[-1]) for i in X_train_hotel_extra])

  X_train_extra = np.array([i[:-1] for i in X_train_extra])
  X_train_hotel_extra = np.array([i[:-1] for i in X_train_hotel_extra])

  X_test_len = np.array([len(i) for i in X_test])
  X_test_extra = X_test[X_test_len>1]

  X_test_hotel_extra_len = np.array([len(i) for i in X_test_hotel_extra])
  X_test_hotel_extra = X_test_hotel_extra[X_test_hotel_extra_len>1]

  y_test_extra = np.array([int(i[-1]) for i in X_test_extra])
  y_test_hotel_extra = np.array([int(i[-1]) for i in X_test_hotel_extra])
  X_test_extra = np.array([i[:-1] for i in X_test_extra])
  X_test_hotel_extra = np.array([i[:-1] for i in X_test_hotel_extra])

  return X_train_extra, y_train_extra, X_test_extra, y_test_extra, y_train_hotel_extra, y_test_hotel_extra, X_train_hotel_extra, X_test_hotel_extra

def partition_arg_topK(matrix, K=10, axis=1):
    """
    perform topK based on np.argpartition
    :param matrix: to be sorted
    :param K: select and sort the top K items
    :param axis: 0 or 1. dimension to be sorted.
    :return:
    """
    a_part = np.argpartition(matrix, K, axis=axis)
    if axis == 0:
        row_index = np.arange(matrix.shape[1 - axis])
        a_sec_argsort_K = np.argsort(matrix[a_part[0:K, :], row_index], axis=axis)
        return a_part[0:K, :][a_sec_argsort_K, row_index]
    else:
        column_index = np.arange(matrix.shape[1 - axis])[:, None]
        a_sec_argsort_K = np.argsort(matrix[column_index, a_part[:, 0:K]], axis=axis)
        return a_part[:, 0:K][column_index, a_sec_argsort_K]


def make_predictions(model, X, X_feat, proba=False, BATCH_SIZE=2**12):
  N = X.shape[0]
  predictions = []
  for i in range(N//BATCH_SIZE+1):
    prediction, _ = model.predict((X[i*BATCH_SIZE:(i+1)*BATCH_SIZE], X_feat[i*BATCH_SIZE:(i+1)*BATCH_SIZE]), batch_size=BATCH_SIZE)
    if proba:
      predictions.append(prediction)
    else:
      label_prediction = partition_arg_topK(-prediction, K=10, axis=1)
      predictions.append(label_prediction)
  return np.vstack(predictions)

def make_prediction(kfold=20, embedding_dim=128, random_state=2020):

  # 分割数据集
  utrip_id_test = data_test.groupby('utrip_id').utrip_id.last().values

  train_user_id = data_train.user_id.unique()
  kf = KFold(n_splits=kfold, random_state=random_state, shuffle=True)

  user_split = []

  for train_index, test_index in kf.split(np.arange(len(train_user_id))):
    user_split.append((train_user_id[train_index], train_user_id[test_index]))

  # fold 0

  all_ids = data_train.groupby('utrip_id').user_id.last().values

  fold = 0
  train_ids = set(list(user_split[fold][0]))
  train_indexs = np.array(list(map(lambda x: True if x in train_ids else False, all_ids)))
  valid_indexs = ~train_indexs

  # 训练模型
  pad_to = 20


  BATCH_SIZE = 2**13

  predictions_valid = []
  predictions_test = []


  for fold in range(kfold):
    start = time.time()
    print('fold %i'%fold)

    train_ids = set(list(user_split[fold][0]))
    train_indexs = np.array(list(map(lambda x: True if x in train_ids else False, all_ids)))
    valid_indexs = ~train_indexs


    X_train, X_test, X_train_hotel, X_test_hotel, X_train0, X_train0_hotel, y_train, y_test, y_train_hotel, y_test_hotel, X_test0 = getdata()
    last_city_test = [int(i[-1]) for i in X_test]

    X_train_extra = X_train.copy()
    X_test_extra = X_test.copy()
    X_train_hotel_extra = X_train_hotel.copy()
    X_test_hotel_extra = X_test_hotel.copy()

    X_train_features, X_test_features = get_additionnal_features(data_train, data_test)
    X_train_features_extand = X_train_features.copy()
    X_train_features_extand = X_train_features_extand[train_indexs]
    X_test_features_extand = X_test_features.copy()

    data_train_extand = data_train.copy()
    data_test_extand = data_test.copy()

    X_valid = X_train[valid_indexs]
    X_valid_features = X_train_features[valid_indexs]
    y_valid = y_train[valid_indexs]
    y_valid_hotel = y_train_hotel[valid_indexs]

    X_train = X_train[train_indexs]
    X_train_features = X_train_features[train_indexs]
    y_train = y_train[train_indexs]
    y_train_hotel = y_train_hotel[train_indexs]

    X_test_origin = X_test.copy()
    X_train_origin = X_train.copy()
    y_train_origin = y_train.copy()
    y_train_hotel_origin = y_train_hotel.copy()


    for i in range(6):
      print('\r epoch %s/6 '%(i+1), end='')
      X_train_extra, y_train_extra, X_test_extra, y_test_extra, y_train_hotel_extra, y_test_hotel_extra, X_train_hotel_extra, X_test_hotel_extra = produce_extra(X_train_extra, X_test_extra, X_train_hotel_extra, X_test_hotel_extra)
      X_train = np.concatenate([X_train, X_train_extra, X_test_extra, ])
      y_train = np.concatenate([y_train, y_train_extra, y_test_extra, ])
      y_train_hotel = np.concatenate([y_train_hotel, y_train_hotel_extra, y_test_hotel_extra, ])

      data_train_extand, data_test_extand = remove_last_row(data_train_extand), remove_last_row(data_test_extand)

      X_train_features_extand_, X_test_features_extand_ = get_additionnal_features(data_train_extand, data_test_extand)
      X_train_features_extand = np.vstack([X_train_features_extand, X_train_features_extand_, X_test_features_extand_])
    print('\r ', end='')

    if fold == 0:
      if os.path.exists('./word2vec_%s.model'%embedding_dim):
        model_sg = Word2Vec.load('./word2vec_%s.model'%embedding_dim)
      else:
        model_sg = Word2Vec(np.concatenate([X_train0, X_test]), size=embedding_dim, window=1, min_count=1, sg=1, sample=1e-3, negative=30, workers=4, iter=30, seed=1)
        model_sg.save('./word2vec_%s.model'%embedding_dim)
      model_emb = model_sg

      embedding_matrix = np.zeros((N_city + 1, embedding_dim))
      for i in range(N_city):
        try:
          embedding_matrix[i+1] = model_emb.wv[str(i)].astype('float32')
        except:
          pass

      embedding_layer = Embedding(N_city + 1,
                      embedding_dim,
                      weights=[embedding_matrix],
                      input_length=pad_to,
                      trainable=False)

    def pad(L, size=embedding_dim):
      n = len(L)
      if n<pad_to:
          L = [0] * (pad_to-n) + [int(i)+1 for i in L]
      else:
        L = [int(i)+1 for i in L][-pad_to:]
      return np.array(L)


    X_train = np.asarray(list(map(lambda x: pad(x), (X_train))) ).astype(int)
    X_valid = np.asarray(list(map(lambda x: pad(x), (X_valid))) ).astype(int)

    X_train_origin = np.asarray(list(map(lambda x: pad(x), (X_train_origin))) ).astype(int)
    X_test_origin = np.asarray(list(map(lambda x: pad(x), (X_test_origin))) ).astype(int)



    def create_model():
      inputs1 = tf.keras.Input(shape=(None,), dtype="int64")
      inputs2 = tf.keras.Input(shape=(X_train_features.shape[1]), dtype="float32")
      
      input3 = inputs2[:,-1]
      input3 = layers.Embedding(20000, 24, trainable=True)(input3)
      
      input2 = layers.BatchNormalization()(inputs2[:,:-1])
      input2 = layers.Dense(128, activation='linear')(input2)
      input2 = layers.BatchNormalization()(input2)
      input2 = layers.ReLU()(input2)
      input2 = layers.Dense(64, activation='linear')(input2)
      input2 = layers.BatchNormalization()(input2)
      input2 = layers.ReLU()(input2)

      input2 = layers.concatenate([input3, input2])


      masked_inputs1 = embedding_layer(inputs1)
      masked_inputs1 = layers.LayerNormalization()(masked_inputs1)

      masked_inputs1 = layers.SpatialDropout1D(0.1)(masked_inputs1)
      x_a = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True, recurrent_initializer="orthogonal", kernel_initializer="glorot_normal",
            ))(masked_inputs1)
      x_a = layers.LayerNormalization()(x_a)
      x_a = layers.SpatialDropout1D(0.1)(x_a)
      x_a = layers.Bidirectional(
        layers.GRU(128, return_sequences=False, recurrent_initializer="orthogonal", kernel_initializer="glorot_normal",
            ))(x_a)
      x_a = layers.LayerNormalization()(x_a)


      x_t = masked_inputs1[:,-1,:]

      x_a = layers.Dropout(0.1)(x_a)
      x_a = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(3e-5), kernel_initializer='uniform')(x_a)
      x_a = layers.LayerNormalization()(x_a)

      x_t = layers.Dropout(0.1)(x_t)
      x_t = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(3e-5), kernel_initializer='uniform')(x_t)
      x_t = layers.LayerNormalization()(x_t)
      
      x_ = layers.concatenate([x_a, x_t])
      x_ = layers.Dropout(0.1)(x_)
      alpha = layers.Dense(512, activation='sigmoid')(x_)

      x = alpha*x_a + (1-alpha)*x_t
      x = layers.LayerNormalization()(x)
      x = layers.concatenate([input2, x])
      x = layers.Dropout(0.5)(x)


      x2 = Dense(N_hotel, activation='softmax', name='country')(x)
      x = Dense(N_city, activation='softmax', name='city', kernel_regularizer=tf.keras.regularizers.L2(1e-4))(x)

      model = keras.Model(inputs=[inputs1, inputs2], outputs=[x, x2])
      return model

    model = create_model()
    if os.path.exists('fold_%s_%s.h5'%(fold, embedding_dim)):
      model.load_weights('fold_%s_%s.h5'%(fold, embedding_dim))
      loss = tf.keras.losses.SparseCategoricalCrossentropy()
      optimizer = AdaBeliefOptimizer(learning_rate=1e-3, epsilon=1e-14, rectify=False, print_change_log = False)
      model.compile(optimizer=optimizer, # tf.keras.optimizers.Adam(),
                        loss=loss,
                        loss_weights=[1, 0.3, 0.3],
                        metrics=['acc',tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)],
                        )
      model.evaluate((X_valid, X_valid_features), (y_valid, y_valid_hotel))
    else:
      loss = tf.keras.losses.SparseCategoricalCrossentropy()

      rlr = ReduceLROnPlateau(monitor = 'val_city_sparse_top_k_categorical_accuracy', factor = 0.3, patience = 3, verbose = 1, 
                              min_delta = 1e-4, mode = 'max', min_lr=1e-5)
      
      ckp_path = 'fold_%s_%s.h5'%(fold, embedding_dim)
      ckp = ModelCheckpoint(ckp_path, monitor = 'city_sparse_top_k_categorical_accuracy', verbose = 0, 
                      save_best_only = True, save_weights_only = True, mode = 'max')
      optimizer = AdaBeliefOptimizer(learning_rate=1e-3, epsilon=1e-14, rectify=False, print_change_log = False)
      model.compile(optimizer=optimizer, # tf.keras.optimizers.Adam(),
                        loss=loss,
                        loss_weights=[1, 0.3, 0.3],
                        metrics=['acc',tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)],
                        
                        )

      history = model.fit((X_train, X_train_features_extand), (y_train, y_train_hotel), 
                          epochs=40, 
                          validation_data=((X_valid, X_valid_features), (y_valid, y_valid_hotel)),
                          shuffle=True,
                          batch_size=BATCH_SIZE,
                          callbacks = [rlr,ckp],
                          verbose=0
                          )
      # model.save_weights('fold_%s_%s.h5'%(fold, embedding_dim))
      model.load_weights('fold_%s_%s.h5'%(fold, embedding_dim))

      
      optimizer = AdaBeliefOptimizer(learning_rate=3e-5, epsilon=1e-14, rectify=False, print_change_log = False)
      model.compile(optimizer=optimizer, # tf.keras.optimizers.Adam(),
                        loss=loss,
                        loss_weights=[1, 0.3, 0.3],
                        metrics=['acc',tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)],
                        )
      ckp_path = 'fold_%s_%s.h5'%(fold, embedding_dim)
      ckp = ModelCheckpoint(ckp_path, monitor = 'city_sparse_top_k_categorical_accuracy', verbose = 0, 
                      save_best_only = True, save_weights_only = True, mode = 'max')
      rlr = ReduceLROnPlateau(monitor = 'val_city_sparse_top_k_categorical_accuracy', factor = 0.3, patience = 3, verbose = 1, 
                              min_delta = 1e-4, mode = 'max', min_lr=1e-6)
      history = model.fit((X_train_origin, X_train_features), (y_train_origin, y_train_hotel_origin, ), 
                          epochs=40, 
                          validation_data=((X_valid, X_valid_features), (y_valid, y_valid_hotel)),
                          shuffle=True,
                          batch_size=BATCH_SIZE,
                          callbacks = [rlr, ckp],
                          verbose=0
                          )
      # model.save_weights('fold_%s_%s.h5'%(fold, embedding_dim))
      model.evaluate((X_valid, X_valid_features), (y_valid, y_valid_hotel))

    # prediction_valid = make_predictions(model, X_valid, X_valid_features, True)
    prediction_test = make_predictions(model, X_test_origin, X_test_features, False)

    # predictions_valid.append(prediction_valid)
    predictions_test.append(prediction_test)

    print(time.time()-start)

  def inverse_trans2(pred):
    shape = pred.shape
    res = le.inverse_transform(pred.reshape(-1))

    return res.reshape(shape)

  np.save(path+'/'+'preds_%sfold_%s.npy'%(fold, embedding_dim), np.array(predictions_test))
  predictions_test_trans = [inverse_trans2(pred) for pred in predictions_test]
  np.save(path+'/'+'preds_%sfold_%s_trans.npy'%(fold, embedding_dim), np.array(predictions_test_trans))
  all_preds = np.hstack(predictions_test_trans)

  return predictions_test

In [ ]:
predictions_test_128 = make_prediction(kfold=20, embedding_dim=224, random_state=2020)


fold 0
 
Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.

Epoch 00031: ReduceLROnPlateau reducing learning rate to 2.700000040931627e-05.

Epoch 00034: ReduceLROnPlateau reducing learning rate to 1e-05.

Epoch 00006: ReduceLROnPlateau reducing learning rate to 8.999999772640877e-06.

Epoch 00009: ReduceLROnPlateau reducing learning rate to 2.6999998226528985e-06.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 1e-06.
342/342 [==============================] - 5s 13ms/step - loss: 3.7850 - city_loss: 3.4150 - country_loss: 0.4368 - city_acc: 0.3466 - city_sparse_top_k_categorical_accuracy: 0.5756 - country_acc: 0.8809 - country_sparse_top_k_categorical_accuracy: 0.9748
2246.9068315029144
fold 1
 
Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.

Epoch 0002

In [ ]:
def inverse_trans2(pred):
  shape = pred.shape
  res = le.inverse_transform(pred.reshape(-1))

  return res.reshape(shape)

np.save('preds_20fold_128.npy', np.array(predictions_test))
predictions_test_trains = [inverse_trans2(pred) for pred in predictions_test]
np.save('preds_20fold_128_trans.npy', np.array(predictions_test_trains))
all_preds = np.hstack(predictions_test_trains)

In [ ]:
from collections import defaultdict

final_pred = []
for index in tqdm(range(len(all_preds))):
  dic = defaultdict(int)
  for i,pred in enumerate(all_preds[index]):
    dic[pred] += 10-(i%10)
  preds = [item[0] for item in sorted(dic.items(), key=lambda item: -item[1])[:4]]
  final_pred.append(preds)

In [ ]:
final_pred = np.array(final_pred)
submission = pd.DataFrame({'utrip_id': data_test.utrip_id.unique(),
                           'city_id_1':final_pred[:,0],
                           'city_id_2':final_pred[:,1],
                           'city_id_3':final_pred[:,2],
                           'city_id_4':final_pred[:,3]
                           })

In [ ]:
submission.to_csv('./submission_team.csv', index=None)
submission.head()

,utrip_id,city_id_1,city_id_2,city_id_3,city_id_4
0,1000066_2,3809,13471,46854,20931
1,1000270_1,29207,22175,13931,53636
2,1000441_1,47759,35160,13260,9485
3,100048_1,13530,57266,26235,45030
4,1000543_1,52818,3145,25473,26951


In [ ]:
def naive_arg_topK(matrix, K=4, axis=1):
    """
    perform topK based on np.argsort
    :param matrix: to be sorted
    :param K: select and sort the top K items
    :param axis: dimension to be sorted.
    :return:
    """
    full_sort = np.argsort(matrix, axis=axis)
    return full_sort.take(np.arange(K), axis=axis)

def partition_arg_topK(matrix, K=4, axis=1):
    """
    perform topK based on np.argpartition
    :param matrix: to be sorted
    :param K: select and sort the top K items
    :param axis: 0 or 1. dimension to be sorted.
    :return:
    """
    a_part = np.argpartition(matrix, K, axis=axis)
    if axis == 0:
        row_index = np.arange(matrix.shape[1 - axis])
        a_sec_argsort_K = np.argsort(matrix[a_part[0:K, :], row_index], axis=axis)
        return a_part[0:K, :][a_sec_argsort_K, row_index]
    else:
        column_index = np.arange(matrix.shape[1 - axis])[:, None]
        a_sec_argsort_K = np.argsort(matrix[column_index, a_part[:, 0:K]], axis=axis)
        return a_part[:, 0:K][column_index, a_sec_argsort_K]


partition_arg_topK(prediction[:10])